# Week 4 Assignment: Trying Out OSMnx Isochrone Mapping
For this assignment, I will be reloading the traffic collision data, my partner and I have cleaned up, to try out the isochrone mapping from our last class. I will also clean up the 

In [1]:
# for spatial data
import geopandas as gpd

# for plotting
import matplotlib.pyplot as plt

# for network analysis
import networkx as nx

# for street network analysis
import osmnx as ox

# for basemaps
import contextily as ctx

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Part 1: Loading cleaned-up bike collision data
The CSV for [LAPD's traffic collision data from 2010 to present](https://data.lacity.org/Public-Safety/Traffic-Collision-Data-from-2010-to-Present/d5tf-ez2w) have been filtered to only include reported traffic collisions involving bikes. For each entry with a lat,long coordinates of "0,0", we have manually gone in to search up lat,long coordinates based on the "Address" (first street) and "Cross Street" (second intersecting street) provided. For entries that have neither lat,long coordinates nor "Address" and "Cross Street" . We created a [Google Doc documenting the process of cleaning up each affected entry](https://docs.google.com/document/d/1Iec81ZXlTOaoUa1vZBHPScEr9pMDE6vaom-cJ_Azbgw/edit?usp=sharing).

The resulting file has also been converted to GEOJSON, so I will load both files to test that they have been loaded properly.

In [ ]:
# configure the place, network type, trip times, and travel speed
address = 'Boyle Heights, Los Angeles, CA, USA'
network_type = 'bike'
trip_times = [5, 10, 15, 20] #in minutes
meters_per_minute = 75 # travel distance per minute